<a href="https://colab.research.google.com/github/adlihs/instatXML/blob/main/Instat_XML_file_process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Process to work with Instat's xml file

## Import libraries

In [ ]:
!pip install pandas_read_xml
!pip install -U pandasql
import pandas_read_xml as pdx 
from pandas_read_xml import flatten, fully_flatten, auto_separate_tables
import os,glob 
import pandas as pd
import time
import numpy as np
from pandasql import sqldf 

## Setup the name of the tournament

In [ ]:
nombre_torneo = "Clausura 2022"

## Setup the google drive folder's path
The idea is to download all the xml from Instat and put all in a google drive folder.

In [ ]:
%cd "/content/drive/My Drive/XML Clausura 2022" #this is an example of my folder's path

## Remove excel files from previous process
The script will generate an excel file for each xml, so the idea is to remove the files from previous process.

In [ ]:
print('Finding and removing old excel files...')
excel_files = []
for file in glob.glob("*.xlsx"):
    excel_files.append(file)

for file in excel_files:
  os.remove(file)

print('done!')

## Let's find all the XML files in the folder

In [ ]:
print('Finding xml files...')
xml_files = []
for file in glob.glob("*.xml"):
    xml_files.append(file)

print('done!')

## Generate an excel file for each xml file

In [ ]:
print('Creating excel files from xmls')
open_file =""
for xml in xml_files:
  # This line will remove the name of the competion from the xml file name
  file_name = xml.replace('--Costa-Rica--Primera-Division','')
  # This line will remove the text '.xml' from the variable 'file_name' 
  file_name = file_name.replace('.xml','')
  # This line will create a dataframe with the xml data
  open_file = pdx.read_xml(xml, ['file', 'ALL_INSTANCES', 'instance'], root_is_rows=False)
  # This line is to create a new column named 'file_name', the column will contain the information of the game
  open_file["file_name"] = file_name
  # This line will generate an excel file
  open_file.to_excel(file_name + ".xlsx")
  #print(open_file.file_name)

print('done!')
open_file

## Merge all excel files in one dataframe

In [ ]:
print('Merging files...')
all_data = pd.DataFrame()
for f in glob.glob("*.xlsx"):
    df = pd.read_excel(f)
    all_data = all_data.append(df,ignore_index=True)
print('done!')
all_data

## Split column `'Label'` by comma `','`

In [ ]:
#Create new dataframe with the new columns from the column 'LABEL'
print('Creating new data frame')
all_data2 = all_data['label'].str.split(',', expand=True)


#Merge original dataframe and new dataframe
print('Merging dataframes')
all_data = pd.concat([all_data, all_data2], axis=1, sort=False)

###REMOVE UNNCESSARY COLUMNS
print('Removing columns')
all_data.drop('label', inplace=True, axis=1)
all_data.drop(0, inplace=True, axis=1)
all_data.drop(2, inplace=True, axis=1)
all_data.drop(4, inplace=True, axis=1)

#### RENAME NEW COLUMNS (Teams, Actions, Time)
print('Reanaming columns')
all_data = all_data.rename(columns={1: "Equipos", 3: "Acciones",5:"Tiempo"})


#### REPLACE CHARACTERS IN COLUMNS EQUIPOS, ACCIONES, TIEMPO
print('Replacing special characters')
import re
chars_to_remove = ['.', '-', '(', ')',']','\'', '','}',':']
regular_expression = '[' + re.escape (''. join (chars_to_remove)) + ']'

all_data['Equipos'] = all_data['Equipos'].str.replace(regular_expression, '', regex=True)
all_data['Acciones'] = all_data['Acciones'].str.replace(regular_expression, '', regex=True)
all_data['Tiempo'] = all_data['Tiempo'].str.replace(regular_expression, '', regex=True)

all_data['Equipos'] = all_data['Equipos'].str.replace('text','')
all_data['Acciones'] = all_data['Acciones'].str.replace('text','')
all_data['Tiempo'] = all_data['Tiempo'].str.replace('text','')


print('done!')


## Preview the dataframe

In [ ]:
all_data.head(20)

## Get new columns from `file_name` column

In [ ]:
#New column 'fecha_partido' (game_date)
print('creating new column fecha_partido')
all_data['fecha_partido'] = all_data['file_name'].str[:10]


#New column 'nombre_torneo' (tournament_name)
print('creating new column nombre_torneo')
all_data['nombre_torneo'] = nombre_torneo

#remove game date from string
print('Removing game date from string')
all_data['file_name'] = all_data['file_name'].str[10:]


#Remove last 20 characters
print('removing unnecessary last 22 last characters')
all_data['file_name'] = all_data['file_name'].str[:-21]

####RENAME COLUMN FILE_NAME
print('Reanaming column')
all_data = all_data.rename(columns={'file_name': "Partido"})

print('done!')

## Preview the dataframe with the new columns

In [ ]:
all_data.head(20)

## Remove white spaces from the column `Acciones` (actions)

In [ ]:
#TRIM spaces
all_data['Acciones'] = all_data['Acciones'].str.strip()
all_data['Equipos'] = all_data['Equipos'].str.strip()

## Setup columns with the `home team name` and the `away team name`

In [ ]:
# Replace score in game description with @
all_data["partido_text"] = all_data["Partido"].replace(to_replace=r'[-]+\d[-]+\d+[-]', value='@', regex=True)

# Create a temp dataframe with columns 'home_team'/'away_team'
localia_df = all_data['partido_text'].str.split('@', expand=True)

# Merge new temp dataframe with all_data dataframe
all_data = pd.concat([all_data, localia_df], axis=1, sort=False)

#Rename new columns
all_data.rename(columns={0: 'equipo_casa', 1: 'equipo_visita'}, inplace=True)

#Drop column 'partido_text'
all_data.drop('partido_text', inplace=True, axis=1)

## Preview the dataframe with the new changes

In [ ]:
all_data.head(20)

## Get columns for: **`Score`**, **`Home and Away goals`**

In [ ]:
# Get the score from column 'Partido' (game)
all_data["marcador"] = all_data['Partido'].str.findall(r'\d[-]+\d+')

#convert column 'marcador' to string
all_data["marcador"] = all_data["marcador"].astype('str') 

# marcador en formato string
all_data["marcador"] = all_data["marcador"].str.slice(start=2, stop=-2)


#New column with the home team goals ('goles_local')
all_data['goles_local'] = all_data["marcador"].str.slice(start=0, stop=1)

#New column with the away team goals ('goles_visita')
all_data['goles_visita'] = all_data["marcador"].str.slice(start=2, stop=3)




## Preview the dataframe with the changes

In [ ]:
all_data.head(20)

## Setup a column with the team that "suffers" the action


In [ ]:
# create a list of our conditions

conditions = [
    (all_data["Equipos"] == all_data["equipo_casa"]),
    (all_data["Equipos"] == all_data["equipo_visita"])]

# create a list of the values we want to assign for each condition
values = [all_data["equipo_visita"], all_data["equipo_casa"]]

# create a new column and use np.select to assign values to it using our lists as arguments
all_data["rival"] = np.select(conditions, values)


## Preview the changes

In [ ]:
all_data.head(30)

## Remove unnecesary columns

In [ ]:
all_data.drop(all_data.columns[0], axis=1,inplace=True)

## Export the consolidated dataframe to Excel file

In [ ]:

print('Creating excel file with the dataframe all_data')
all_data.to_excel('UNAFUT_'+nombre_torneo+'.xlsx') # YOU CAN SETUP THE FILE NAME THAT YOU WANT
print('done!')

## Delete all excel files except the file created in the previous step

In [ ]:
print('Detecting all .xlsx files')
excel_files = []
for file in glob.glob("*.xlsx"):
    excel_files.append(file)

print('Removing all excel files except UNAFUT_' + nombre_torneo +'.xlsx')
for file in excel_files:
  if file != ('UNAFUT_' + nombre_torneo + '.xlsx') : # HERE CHANGE AND PUT THE FILE NAME YOU PUT IN THE PREVIOUS STEP
    os.remove(file)

print('All process is done!')